In [1]:
import config
import models
import tensorflow as tf
import numpy as np
import os


# Run TransE To Create initial embeddings

In [ ]:
"""
Method:

Run the normal transe example (example_train_transe.py)
Write the embeddings as a file that can be read
use embeddings to initialize embedding layer in TransE_freeze.py

Append random embeddings for new entities and relations
set config freeze_train_embeddings = true
figure out how to update only the embeddings for a certain set of indices
figure out how to make sure that we only see examples using new items to speed convergence
compare new+old embeddings
"""
os.environ['CUDA_VISIBLE_DEVICES']='7'
#Input training files from benchmarks/FB15K/ folder.
con = config.Config()
#True: Input test files from the same folder.
con.set_in_path("./benchmarks/FB15K/")
con.set_test_link_prediction(True)
con.set_test_triple_classification(True)
con.set_work_threads(8)
con.set_train_times(10)
con.set_nbatches(20)
con.set_alpha(0.001)
con.set_margin(1.0)
con.set_bern(0)
con.set_dimension(100)
con.set_ent_neg_rate(1)
con.set_rel_neg_rate(0)
con.set_opt_method("SGD")
con.set_freeze_train_embeddings(True)
con.set_ent_embedding_initializer("new path")
con.set_rel_embedding_initializer("new _ path")

In [ ]:
#Models will be exported via tf.Saver() automatically.
con.set_export_files("./res/model.vec.tf", 0)
#Model parameters will be exported to json files automatically.
con.set_out_files("./res/embedding.vec.json")
#Initialize experimental settings.
con.init()

In [ ]:

#Set the knowledge embedding model
con.set_model(models.TransE_freeze)


In [ ]:
#Train the model.
con.run()

In [ ]:
# This method predicts the top k head entities given tail entity and relation.
# t (int): tail entity id
# r (int): relation id
# k (int): top k head entities
con.predict_head_entity(152, 9, 5)
con.predict_tail_entity(151, 9, 5)
con.predict_relation(151, 152, 5)

# head, tail, relation
con.predict_triple(151, 152, 9)
con.predict_triple(151, 152, 8)

# Run new example using Embeddings produced in above step

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='7'
#Input training files from benchmarks/FB15K/ folder.
con = config.Config()
#True: Input test files from the same folder.
con.set_in_path("./benchmarks/FB15K/")
con.set_test_link_prediction(True)
con.set_test_triple_classification(True)
con.set_work_threads(8)
con.set_train_times(10)
con.set_nbatches(20)
con.set_alpha(0.001)
con.set_margin(1.0)
con.set_bern(0)
con.set_dimension(100)
con.set_ent_neg_rate(1)
con.set_rel_neg_rate(0)
con.set_opt_method("SGD")
con.set_freeze_train_embeddings(True)
con.set_ent_embedding_initializer("./res/embedding.vec.json")
con.set_rel_embedding_initializer("./res/embedding.vec.json")

In [3]:
#Models will be exported via tf.Saver() automatically.
con.set_export_files("./res/model_new.vec.tf", 0)
#Model parameters will be exported to json files automatically.
con.set_out_files("./res/embedding_new.vec.json")
#Initialize experimental settings.
con.init()
#Set the knowledge embedding model
con.set_model(models.TransE_freeze)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

LOOK RIGHT HERE!!!!!
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [4]:
#Train the model.
con.run()

Epoch: 0, loss: 40036.40954589844, time: 1.1920928955078125e-06
Epoch: 1, loss: 38376.0673828125, time: 0.0
Epoch: 2, loss: 37797.5771484375, time: 9.5367431640625e-07
Epoch: 3, loss: 36010.30334472656, time: 1.1920928955078125e-06
Epoch: 4, loss: 34296.89697265625, time: 1.1920928955078125e-06
Epoch: 5, loss: 33197.309326171875, time: 9.5367431640625e-07
Epoch: 6, loss: 32095.247314453125, time: 1.1920928955078125e-06
Epoch: 7, loss: 31608.9091796875, time: 9.5367431640625e-07
Epoch: 8, loss: 30047.505981445312, time: 0.0
Epoch: 9, loss: 28439.334228515625, time: 9.5367431640625e-07


In [ ]:
import json
f = open("./res/embedding.vec.json", "r")
embeddings = json.loads(f.read())

In [ ]:
ent_embeddings = embeddings["ent_embeddings"]
rel_embeddings = embeddings["rel_embeddings"]

In [ ]:
print(len(ent_embeddings))
print(len(rel_embeddings))

In [ ]:
rel_embeddings[0]

In [ ]:
initial = tf.constant_initializer(ent_embeddings)